# TA-Lib Candlestick Scanner + Backtester (Swing Trading) — v4

### What's fixed
- Scanner returns the **most recent** valid pattern inside `recent_window` across *all* patterns.
- Trend filter is evaluated **at the signal bar**.
- Scan output includes **`Pattern Time`** for direct chart verification.


## 1) Setup

In [1]:
import sys, subprocess, importlib

def ensure(pkg, pip_name=None):
    try:
        importlib.import_module(pkg)
        print(f"OK: {pkg}")
    except ImportError:
        name = pip_name or pkg
        print(f"Installing {name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", name])

for p in ["yfinance", "pandas", "numpy", "matplotlib"]:
    ensure(p)

# TA-Lib
try:
    import talib as ta
    print("OK: TA-Lib available")
except Exception:
    print("TA-Lib not found, attempting pip install...")
    ensure("talib", "TA-Lib")
    import talib as ta

import pandas as pd
import numpy as np
import yfinance as yf
import math
import json
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 160)

OK: yfinance
OK: pandas
OK: numpy
OK: matplotlib
OK: TA-Lib available


## 2) Parameters

In [2]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
symbols = ["SIEMENS.NS"]       # Replace with your tickers
timeframe = "1d"               # '1d' or '4h'
lookback_days = 1825

recent_window = 3              # Scan last N candles for a fresh signal
use_trend_filter_scanner = True

# Backtest params (optional to use later)
signal_set = "bullish"
use_trend_filter_backtest = True
allow_shorts = False

atr_period = 14
atr_sl_mult = 2.0
atr_tp_mult = 2.0

initial_equity = 100000.0
risk_per_trade_pct = 1.0
fees_bps_per_side = 10.0
slippage_bps_per_side = 5.0
max_holding_bars = 20
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

assert timeframe in ("1d", "4h")
assert signal_set in ("bullish", "bearish", "both")

## 3) Helpers & TA-Lib Signals

In [3]:
def yf_params(timeframe: str, lookback_days: int):
    if timeframe == "1d":
        period = f"{min(max(lookback_days, 2), 3650)}d"
        interval = "1d"
    else:
        period = "60d"
        interval = "4h"
    return dict(period=period, interval=interval)

def fetch_ohlc(ticker: str, period: str, interval: str) -> pd.DataFrame:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        try:
            df = df.xs(ticker, axis=1, level=1)
        except Exception:
            pass
    df = df.rename(columns={c: c.title() for c in df.columns}).dropna()
    return df

def series_slope(series: pd.Series, window: int = 5) -> pd.Series:
    return series.rolling(window).apply(lambda x: np.polyfit(np.arange(len(x)), x, 1)[0], raw=True)

def in_downtrend_series(df: pd.DataFrame, window: int = 5) -> pd.Series:
    close = df['Close']; sma20 = close.rolling(20).mean(); sl = series_slope(close, window)
    return (sl < 0) & (close < sma20)

def in_uptrend_series(df: pd.DataFrame, window: int = 5) -> pd.Series:
    close = df['Close']; sma20 = close.rolling(20).mean(); sl = series_slope(close, window)
    return (sl > 0) & (close > sma20)

In [4]:
BULLISH_NAMES = ["Bullish Engulfing","Morning Star","Hammer","Piercing Line","Three White Soldiers","Bullish Harami"]
BEARISH_NAMES = ["Bearish Engulfing","Evening Star","Shooting Star","Dark Cloud Cover","Three Black Crows","Bearish Harami"]

PATTERN_CONTEXT = {
    "Bullish Engulfing": "End of a downtrend; second candle's body engulfs prior body — bullish reversal potential.",
    "Morning Star": "Downtrend: red, small 'star', strong green — reversal.",
    "Hammer": "Downtrend: long lower shadow rejects lows — reversal potential.",
    "Piercing Line": "Downtrend: gap-down then close above prior midpoint — reversal.",
    "Three White Soldiers": "Three strong advances after weakness — bullish continuation/reversal.",
    "Bullish Harami": "Downtrend: small green within a large red — reversal potential.",
    "Bearish Engulfing": "Uptrend: large red engulfs prior body — bearish reversal potential.",
    "Evening Star": "Uptrend: green, small 'star', strong red — reversal.",
    "Shooting Star": "Uptrend: long upper shadow rejects highs — reversal potential.",
    "Dark Cloud Cover": "Uptrend: gap-up then close below prior midpoint — reversal.",
    "Three Black Crows": "Three strong declines after strength — bearish continuation/reversal.",
    "Bearish Harami": "Uptrend: small red within a large green — reversal potential.",
}

In [5]:
import talib as ta
import pandas as pd
import numpy as np

def talib_signals(df: pd.DataFrame) -> pd.DataFrame:
    o = df['Open'].values.astype(float)
    h = df['High'].values.astype(float)
    l = df['Low'].values.astype(float)
    c = df['Close'].values.astype(float)

    out = {}
    out["Bullish Engulfing"] = ta.CDLENGULFING(o,h,l,c)
    out["Bearish Engulfing"] = out["Bullish Engulfing"]

    out["Morning Star"] = ta.CDLMORNINGSTAR(o,h,l,c)
    out["Evening Star"] = ta.CDLEVENINGSTAR(o,h,l,c)

    out["Hammer"] = ta.CDLHAMMER(o,h,l,c)
    out["Shooting Star"] = ta.CDLSHOOTINGSTAR(o,h,l,c)

    out["Piercing Line"] = ta.CDLPIERCING(o,h,l,c)
    out["Dark Cloud Cover"] = ta.CDLDARKCLOUDCOVER(o,h,l,c)

    out["Three White Soldiers"] = ta.CDL3WHITESOLDIERS(o,h,l,c)
    out["Three Black Crows"] = ta.CDL3BLACKCROWS(o,h,l,c)

    out["Bullish Harami"] = ta.CDLHARAMI(o,h,l,c)
    out["Bearish Harami"] = out["Bullish Harami"]

    sig = pd.DataFrame(out, index=df.index)
    for col in sig.columns:
        sig[col] = np.where(sig[col] > 0, 1, np.where(sig[col] < 0, -1, 0))
    return sig

## 4) Fixed Scanner

In [6]:
def find_latest_signal(df: pd.DataFrame, recent_window: int = 3, use_trend=True):
    if len(df) < 30:
        return None

    sig_all = talib_signals(df)

    # Precompute trend *series* (align with df)
    down = in_downtrend_series(df).fillna(False)
    up = in_uptrend_series(df).fillna(False)

    start = max(0, len(df) - recent_window)
    best = None  # dict with pattern, side, index

    # Iterate all patterns and keep the most recent valid one
    for name in (BULLISH_NAMES + BEARISH_NAMES):
        vals = sig_all[name].values
        for i in range(len(vals)-1, start-1, -1):
            v = vals[i]
            if v == 0:
                continue
            side = "Bullish" if name in BULLISH_NAMES else "Bearish"
            # Harami sign check
            if name == "Bullish Harami" and v < 0: 
                continue
            if name == "Bearish Harami" and v > 0: 
                continue
            # Trend filter at the signal bar
            if use_trend:
                if side == "Bullish" and not bool(down.iloc[i]):
                    continue
                if side == "Bearish" and not bool(up.iloc[i]):
                    continue
            if (best is None) or (i > best['index']):
                best = dict(pattern=name, side=side, index=i, time=df.index[i], context=PATTERN_CONTEXT[name])
    return best

def run_scanner(symbols, timeframe, lookback_days, recent_window, use_trend):
    yfp = yf_params(timeframe, lookback_days)
    rows = []
    for sym in symbols:
        try:
            df = fetch_ohlc(sym, **yfp)
            if df.empty:
                rows.append(dict(symbol=sym, pattern="No clear setup", side="", pattern_time="", context="No data"))
                continue
            sig = find_latest_signal(df, recent_window=recent_window, use_trend=use_trend)
            if sig is None:
                rows.append(dict(symbol=sym, pattern="No clear setup", side="", pattern_time="", context="—"))
            else:
                rows.append(dict(symbol=sym, pattern=sig['pattern'], side=sig['side'], pattern_time=str(sig['time']), context=sig['context']))
        except Exception as e:
            rows.append(dict(symbol=sym, pattern="Error", side="", pattern_time="", context=str(e)))
    res = pd.DataFrame(rows, columns=["symbol","pattern","side","pattern_time","context"])
    return res

scan_results = run_scanner(symbols, timeframe, lookback_days, recent_window, use_trend_filter_scanner)
scan_results = scan_results.rename(columns={
    "symbol":"Stock Symbol",
    "pattern":"Pattern Detected",
    "side":"Bullish/Bearish",
    "pattern_time":"Pattern Time",
    "context":"When this pattern usually forms"
})
scan_results

,Stock Symbol,Pattern Detected,Bullish/Bearish,Pattern Time,When this pattern usually forms
0,SIEMENS.NS,Bearish Engulfing,Bearish,2025-09-04 00:00:00,Uptrend: large red engulfs prior body — bearis...


In [7]:
# Save scan results with pattern time
scan_csv = f"talib_candlestick_scan_{timeframe}.csv"
scan_results.to_csv(scan_csv, index=False)
print(f"Saved scan CSV: {scan_csv}")

Saved scan CSV: talib_candlestick_scan_1d.csv
